#### Análisis y Procesamiento de Señales

# Tarea Semanal N°9 
#### Eugenio Briamonte
#### Federico Dunel


#### Introduccion

Cuando la información útil de una señal está en la forma de ésta y es necesario preservarla el filtrado no lineal es la herramienta adecuada. Los filtrados que realizamos sobre señales de ECG consisten en aplicar un filtro de mediana, estimar ruido de baja frecuencia mediante splines cubicos y aplicar un filtro adaptado para detectar latidos. 
El filtro de mediana funciona recorriendo toda la señal aplicando una ventana y seleccionando la mediana (El valor intermedio entre el maximo y minimo local) de ese entorno. Luego cambia el valor de la muestra en la que se encuentra por la mediana de su entorno. Es un filtro util para registrar los cambios mas lentos y prolongados e ignorar los picos o cambios abruptos.

El filtrado por splines cubicos consiste en ubicar puntos de baja actividad cardíaca en el ecg y conectarlos entre si por una función estimadora, un polinomio interpolador de orden 3 entre cada punto. La idea es aproximar el movimiento de la linea de base de la señal con este polinomio interpolador y luego removerlo de la señal, eliminando este ruido de baja frecuencia.

El filtro adaptado utiliza una señal de morfologia similar a una estructura buscada en la señal a analizar y las correlaciona, dandonos informacion util sobre la similitud entre secciones de la señal y la estructura buscada.

#### Desarrollo

Aplicamos el filtro de mediana para encontrar el ruido de linea de base (Ruido de baja frecuencia y alta potencia que afecta al suelo sobre el que se registra el ECG, dándole una forma levemente senoidal a toda la señal). Usando la funcion median.filter de la libreria scipy.signal, primero buscando la media en vecindades de 200 muestras y luego repitiendo el procedimiento sobre vecindades de 600 muestras. Esto nos permitió 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import scipy.io as sio
import scipy.ndimage as nd 
import scipy.interpolate as inter

#------------------------------------------------------------------------------
# Lectura de ECG #
#------------------------------------------------------------------------------


fs_ecg = 1000 # Hz

#------------------------------------------------------------------------------
# ECG con ruido
#------------------------------------------------------------------------------
# para listar las variables que hay en el archivo
sio.whosmat('ecg.mat')
mat_struct = sio.loadmat('ecg.mat')

ecg_one_lead = mat_struct['ecg_lead']

N = np.arange(1129116)

hb_1 = mat_struct['heartbeat_pattern1']
hb_2 = mat_struct['heartbeat_pattern2']
qrs = mat_struct['qrs_pattern1']
ecg_cortado = ecg_one_lead[5000:12000]
#------------------------------------------------------------------------------
# Filtro de mediana
#------------------------------------------------------------------------------
primer_filt = nd.median_filter(ecg_cortado, 200)
Estimacion_ruido_lb = nd.median_filter(primer_filt, 600)

ECG_sin_Inter = ecg_cortado - Estimacion_ruido_lb

plt.figure(1)
plt.plot(ecg_cortado)
plt.plot(ECG_sin_Inter)


Utilizando los datos proporcionados de latidos pudimos localizar las zonas de baja actividad cardiaca desplazandonos algunas muestras hacia el intervalo entre la onda P y la onda Q. Haciendo esto, pudimos localizar un conjunto de muestras sobre el ECG para darle a la función CubicSpline de la libreria scipy.interpolate y armar nuestra estimación del ruido de linea de base. Luego resampleamos esa señal a la misma frecuencia que la señal del ecg para hacerlas coincidir y eliminamos los elementos del Ruido del ECG, obteniendo una señal mucho mas estable. 

In [ ]:
#%%
#------------------------------------------------------------------------------
# Filtrado con Spline Cubico
#------------------------------------------------------------------------------
latidos = mat_struct['qrs_detections'].reshape(1903) #Coordenadas en muestra de latidos
IsoElec = latidos - 400 #Coordenadas en muestras de ptos isoelectricos

Spline_Ruido = inter.CubicSpline(IsoElec,ecg_one_lead[IsoElec])

Ruido_sp = Spline_Ruido(N)
Ecg_filtrado = ecg_one_lead - Ruido_sp

plt.figure(6)
plt.plot(Ecg_filtrado[5000:12000])
plt.plot(ecg_one_lead[5000:12000])

Utilizando un modelo de complejo QRS, realizamos una correlación espejada entre la señal del ECG y el modelo y registramos todos los puntos donde la correlación se hace 0. En todos estos puntos sabemos que la señal y el modelo son muy similares, y por lo tanto inferimos que ahí ocurre un latido. Aplicamos una mascara booleana a la función de correlación para obtener las posiciones iguales a 0, y luego multiplicamos esa mascara por la señal original. De esta manera aislamos las posiciones donde aparece un latido y eliminamos todas las que no. Esto nos permite utilizar la función signal.find_peaks, que nos permite parametrizar picos y buscarlos sobre una señal con un margen o umbral de aprobación. Este método nos permitió encontrar 1662 latidos sobre un total de 1903, pero debido a la poca precisión de nuestro método solo 6 de estos valores coinciden perfectamente con un latido. El resto caen en un rango cercano pero no podemos decir con certeza cuales son falsos negativos y cuales verdaderos positivos. 


In [ ]:
#------------------------------------------------------------------------------
# Filtro Adaptado
#------------------------------------------------------------------------------

#Correlaciono la señal del ecg con un patron conocido sin ruido, me quedo con 
#los valores superiores a algun umbral para anular el ruido 

Correlate = sig.correlate(ecg_one_lead, hb_2, 'same')
Masc = (Correlate == 0)  
Lats = sig.find_peaks((ecg_one_lead * Masc).reshape(1129116), 150, distance = 350)
plt.plot(Correlate)
plt.plot(Lats)



#### Autoevaluación

Esta tarea fue la que menor tiempo me tomó implementar en un código, siento que finalmente entiendo lo que piden de nosotros cuando leo una consigna. Me encuentro satisfecho con lo aprendido y me alegra enormemente haber seguido hasta el final (Con las inconsistencias típicas de la vida universitaria). Este trabajo fue realizado enteramente con Python, mis conocimientos de analisis de señales y GeminiAI para consultar algunas librerias oscuras de Scipy. Muchos conceptos de esta materia y en especial de este trabajo los entendí mucho mejor al trabajarlos en el módulo de imagenes, el concepto de la no linealidad de un filtro es algo que me hubiera costado mucho mas de no existir dicho módulo. Gran detalle pedirnos buscar en Wikipedia, es un concepto un poco vintage a la fecha considerando los cambios habidos. Me fascinó aprender sobre las aplicaciones del procesamiento de imagenes y señales digitales, definitivamente es un campo mucho mas abarcativo del que me imaginé al comenzar la materia y puedo ver claramente el uso que le daré en el futuro. Este es el útlimo trabajo que me quedaba por entregar, gracias por acompañarme en este cuatrimestre.